In [9]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

import torch
from allennlp.nn import util

In [2]:
import sys
sys.path.insert(0, "../../gector")
sys.path.insert(0, "../../")

In [3]:
from gector.gec_model import GecBERTModel

In [4]:
vocab_path = "../../data/output_vocabulary"
model_paths = "../../models/Exp_005_roberta_base_coldstep_2_fixed_vocab/best.th"
model_name = 'roberta'

model_1 = GecBERTModel(vocab_path=vocab_path,
                     model_paths=[model_paths],
                     max_len=50, 
                     min_len=3,
                     iterations=5,
                     min_error_probability=0.0,
                     min_probability=0.0,
                     lowercase_tokens=0,
                     model_name= model_name,
                     special_tokens_fix=1,
                     log=False,
                     confidence=0,
                     is_ensemble=0,
                     weigths=None,
                     use_cpu=False)

In [5]:
vocab_path = "../../data/output_vocabulary"
model_paths = "../../models/Exp_049_2_roberta_base_stage_3_gold/model_state_epoch_3.th"
model_name = 'roberta'

model_2 = GecBERTModel(vocab_path=vocab_path,
                     model_paths=[model_paths],
                     max_len=50, 
                     min_len=3,
                     iterations=5,
                     min_error_probability=0.0,
                     min_probability=0.0,
                     lowercase_tokens=0,
                     model_name= model_name,
                     special_tokens_fix=1,
                     log=False,
                     confidence=0,
                     is_ensemble=0,
                     weigths=None,
                     use_cpu=False)

In [6]:
def get_embedings_for_batch(words_batch, model):
    batch = model.preprocess(words_batch)
    batch = util.move_to_device(batch[0].as_tensor_dict(), 0 if torch.cuda.is_available() else -1)
    embed = model.models[0].text_field_embedder(batch['tokens'])
    tensors = []
    for i in range(len(words_batch)):
        tensors.append(embed[i][batch['tokens']['mask'][i]==1].mean(dim=0).cpu().detach().numpy())
    return tensors

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [8]:
import pickle 
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

import os
import glob
def read_lines(fn):
    if not os.path.exists(fn):
        return []
    with open(fn, 'r', encoding='utf-8') as f:
        text = f.read()
    lines = text.split("\n")
    if lines[-1] == '':
        return lines[:-1]
    else:
        return lines

In [11]:
from tqdm.auto import tqdm

In [12]:
def get_embedings_for_text(src_text, model, batch_size=32):
    embedings = []
    batch = []
    for sent in tqdm(src_text):
        batch.append(sent.split())
        if len(batch) == batch_size:
            batch_embed = get_embedings_for_batch(batch, model)
            embedings.extend(batch_embed)
            batch = []
    batch_embed = get_embedings_for_batch(batch, model)
    embedings.extend(batch_embed)
    return embedings

### Fce

In [44]:
fce_texts_src = read_lines("../../data_parallel/fce/fce_train_src")

In [45]:
fce_embed_1 = get_embedings_for_text(fce_texts_src, model_1, batch_size=32)

In [46]:
fce_embed_2 = get_embedings_for_text(fce_texts_src, model_2, batch_size=32)

In [47]:
fce_cos = np.diag(cosine_similarity(fce_embed_1,fce_embed_2))

In [48]:
import pickle
with open("fce_embed_1_large.pickle", "wb") as f:
    pickle.dump(fce_embed_1, f)

In [49]:
with open("fce_embed_2_large.pickle", "wb") as f:
    pickle.dump(fce_embed_2, f)

In [50]:
with open("fce_cos_large.pickle", "wb") as f:
    pickle.dump(fce_cos, f)

for base

In [67]:
with open("fce_embed_1.pickle", "rb") as f:
    fce_embed_1 = pickle.load(f)

with open("fce_embed_2.pickle", "rb") as f:
    fce_embed_2 = pickle.load(f)

In [68]:
fce_cos = np.diag(cosine_similarity(fce_embed_1,fce_embed_2))

In [70]:
with open("fce_cos.pickle", "wb") as f:
    pickle.dump(fce_cos, f)

In [18]:
fce_cos.min()

0.7934608

In [19]:
fce_cos.max()

0.9866495

In [20]:
fce_cos.mean()

0.9742297

for large

In [51]:
fce_cos.min()

0.93526495

In [52]:
fce_cos.max()

0.9965112

In [53]:
fce_cos.mean()

0.9899379

### Nucle

In [54]:
nucle_texts_src = read_lines("../../data_parallel/nucle/nucle_src")

In [55]:
nucle_embed_1 = get_embedings_for_text(nucle_texts_src, model_1, batch_size=32)

In [56]:
nucle_embed_2 = get_embedings_for_text(nucle_texts_src, model_2, batch_size=32)

In [57]:
with open("nucle_embed_2_large.pickle", "wb") as f:
    pickle.dump(nucle_embed_2, f)

In [58]:
with open("nucle_embed_1_large.pickle", "wb") as f:
    pickle.dump(nucle_embed_1, f)

In [59]:
nucle_cos = np.diag(cosine_similarity(nucle_embed_1,nucle_embed_2))

In [60]:
with open("nucle_cos_large.pickle", "wb") as f:
    pickle.dump(nucle_cos, f)

In [31]:
nucle_cos.min()

0.80371165

In [32]:
nucle_cos.max()

0.98458177

In [33]:
nucle_cos.mean()

0.9692955

### Lang8

In [ ]:
lang8_texts_src = read_lines("../../data_parallel/lang8/lang8_src")

In [35]:
lang8_embed_1 = get_embedings_for_text(lang8_texts_src, model_1, batch_size=32)

In [36]:
lang8_embed_2 = get_embedings_for_text(lang8_texts_src, model_2, batch_size=32)

In [37]:
with open("lang8_embed_2.pickle", "wb") as f:
    pickle.dump(lang8_embed_2, f)

In [38]:
with open("lang8_embed_1.pickle", "wb") as f:
    pickle.dump(lang8_embed_1, f)

In [40]:
#lang8_cos = np.diag(cosine_similarity(lang8_embed_1,lang8_embed_2))

In [ ]:
with open("lang8_cos.pickle", "wb") as f:
    pickle.dump(lang8_cos, f)

In [ ]:
#lang8_cos.min()

In [ ]:
#lang8_cos.max()

In [ ]:
#lang8_cos.mean()

In [61]:
lang8_embed_1_large = get_embedings_for_text(lang8_texts_src, model_1, batch_size=32)

In [63]:
lang8_embed_2_large = get_embedings_for_text(lang8_texts_src, model_2, batch_size=32)

In [64]:
with open("lang8_embed_2_large.pickle", "wb") as f:
    pickle.dump(lang8_embed_2_large, f)

In [62]:
with open("lang8_embed_1_large.pickle", "wb") as f:
    pickle.dump(lang8_embed_1_large, f)

In [ ]:
lang8_cos_large = np.diag(cosine_similarity(lang8_embed_1_large,lang8_embed_2_large))

In [ ]:
with open("lang8_cos_large.pickle", "wb") as f:
    pickle.dump(lang8_cos_large, f)

In [43]:
del lang8_embed_2
del lang8_embed_1

del fce_embed_2
del fce_embed_1

del nucle_embed_2
del nucle_embed_1

del fce_cos
del nucle_cos
#del lang8_cos

### Try large models

In [41]:
vocab_path = "../../data/output_vocabulary"
model_paths = "../../models/Exp_008_roberta_large/best.th"
model_name = 'roberta-large'

model_1 = GecBERTModel(vocab_path=vocab_path,
                     model_paths=[model_paths],
                     max_len=50, 
                     min_len=3,
                     iterations=5,
                     min_error_probability=0.0,
                     min_probability=0.0,
                     lowercase_tokens=0,
                     model_name= model_name,
                     special_tokens_fix=1,
                     log=False,
                     confidence=0,
                     is_ensemble=0,
                     weigths=None,
                     use_cpu=False)

In [42]:
vocab_path = "../../data/output_vocabulary"
model_paths = "../../models/Exp_037_roberta_large_st3/model_state_epoch_1.th"
model_name = 'roberta-large'

model_2 = GecBERTModel(vocab_path=vocab_path,
                     model_paths=[model_paths],
                     max_len=50, 
                     min_len=3,
                     iterations=5,
                     min_error_probability=0.0,
                     min_probability=0.0,
                     lowercase_tokens=0,
                     model_name= model_name,
                     special_tokens_fix=1,
                     log=False,
                     confidence=0,
                     is_ensemble=0,
                     weigths=None,
                     use_cpu=False)

In [12]:
# from transformers import pipeline, RobertaForMaskedLM, RobertaTokenizer

# model = RobertaForMaskedLM.from_pretrained("youscan/ukr-roberta-base")
# tokenizer = RobertaTokenizer.from_pretrained("youscan/ukr-roberta-base")

In [10]:
#os.listdir('../../models/Exp_005_roberta_base_coldstep_2_fixed_vocab/')

In [9]:
from sentence_transformers import SentenceTransformer

In [3]:
from transformers import AutoTokenizer, AutoModel


In [6]:
# from transformers import pipeline, RobertaForMaskedLM, RobertaTokenizer

# model = RobertaForMaskedLM.from_pretrained("ukr-roberta-base")
# tokenizer = RobertaTokenizer.from_pretrained("ukr-roberta-base")

In [13]:
tokenizer = AutoTokenizer.from_pretrained("youscan/ukr-roberta-base")

In [106]:
# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
# outputs = model(input_ids)
# last_hidden_states = outputs[0]

In [8]:
#sorted(os.listdir("../models/Exp_046_roberta_base_stage_2_new/"))